<a href="https://colab.research.google.com/github/Arindam75/graph_networks/blob/main/06_graph_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.4 MB/s eta 0:00:00


In [3]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
from torch.nn import Linear, Dropout

# Import dataset from PyTorch Geometric
dataset = Planetoid(root=".", name="Cora")
data = dataset[0]

In [49]:
def accuracy(y_pred, y_true):
  return torch.sum(y_pred == y_true) / len(y_true)

class GAT(torch.nn.Module):
  def __init__(self, dim_in, dim_h, dim_out, heads=8):
    super().__init__()

    self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
    self.gat2 = GATv2Conv(dim_h*heads, dim_out,heads=1)

  def forward(self, x, edge_index):
    h = F.dropout(x, p=0.6, training=self.training)
    h = self.gat1(h, edge_index)
    h = F.elu(h)
    h = F.dropout(h, p=0.6, training=self.training)
    h = self.gat2(h, edge_index)
    return F.log_softmax(h, dim=1)

class GAT(torch.nn.Module):
  def __init__(self, dim_in, dim_h, dim_out, heads=8):
      super().__init__()
      self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
      self.gat2 = GATv2Conv(dim_h*heads, dim_out, heads=1)

  def forward(self, x, edge_index):
      h = F.dropout(x, p=0.6, training=self.training)
      h = self.gat1(h, edge_index)
      h = F.elu(h)
      h = F.dropout(h, p=0.6, training=self.training)
      h = self.gat2(h, edge_index)
      return F.log_softmax(h, dim=1)

  def fit(self, data , epochs):

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=0.01)

    self.train()
    for epoch in range(epochs):
      optimizer.zero_grad()
      out = self(data.x, data.edge_index)

      loss = criterion(out[data.train_mask], data.y[data.train_mask])
      acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
      loss.backward()
      optimizer.step()

      if epoch%20==0:
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
        print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')

  @torch.no_grad()
  def test(self, data):
    self.eval()
    out = self(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

In [50]:
gat = GAT(dataset.num_features, 32, dataset.num_classes)
print(gat)

GAT(
  (gat1): GATv2Conv(1433, 32, heads=8)
  (gat2): GATv2Conv(256, 7, heads=1)
)


In [51]:
# Train
gat.fit(data, epochs=100)

# Test
acc = gat.test(data)
print(f'GAT test accuracy: {acc*100:.2f}%')

Epoch   0 | Train Loss: 1.949 | Train Acc: 10.71% | Val Loss: 1.95 | Val Acc: 13.60%
Epoch  20 | Train Loss: 0.185 | Train Acc: 100.00% | Val Loss: 0.85 | Val Acc: 75.60%
Epoch  40 | Train Loss: 0.183 | Train Acc: 96.43% | Val Loss: 0.92 | Val Acc: 72.40%
Epoch  60 | Train Loss: 0.197 | Train Acc: 98.57% | Val Loss: 0.90 | Val Acc: 71.60%
Epoch  80 | Train Loss: 0.171 | Train Acc: 98.57% | Val Loss: 0.85 | Val Acc: 72.00%
GAT test accuracy: 79.50%


In [52]:
acc = gat.test(data)
print(f'GAT test accuracy: {acc*100:.2f}%')

GAT test accuracy: 79.50%
